In [64]:
from langchain.document_loaders import JSONLoader
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

from pathlib import Path
from pprint import pprint

import json

load_dotenv()

True

## Load Records

In [2]:
coords_path = "/Users/davidrose/git/chatlas/data/location_history/Records.json"

with open(coords_path) as f:
    data = json.load(f)

In [22]:
df_loc = pd.json_normalize(data["locations"])
df_loc["timestamp"] = pd.to_datetime(df_loc["timestamp"])
df = df_loc.sample(10_000)

# Function to extract the top activity
def get_top_activity(activities):
    # If list
    if isinstance(activities, list):
        if not activities:  # Check for empty list
            return None
    
        # Good case
        top_activities = []
        for i in activities:
            top = max(i["activity"], key=lambda x: x["confidence"])
            top_activities.append(top)
        single_top_activity = max(top_activities, key=lambda x: x["confidence"])
        return single_top_activity
    
    # If nan float
    elif pd.isna(activities):
        return None

# Apply the function to each row and store in a new column
df["top_activity"] = df["activity"].apply(get_top_activity)
df["confidence"] = df["top_activity"].apply(lambda x: x["confidence"] if x else None)
df["top_activity"] = df["top_activity"].apply(lambda x: x["type"] if x else None)

# Drop the old activity column
df.drop(columns=["activity"], inplace=True)

# lowercase all columns
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [23]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

In [24]:
agent.run("What are my top 3 activities?")



> Entering new AgentExecutor chain...
Thought: I need to find the top 3 activities in the dataframe
Action: python_repl_ast
Action Input: df['top_activity'].value_counts().head(3)
Observation: still      1965
tilting    1012
unknown     301
Name: top_activity, dtype: int64
Thought: I now know the top 3 activities
Final Answer: still, tilting, and unknown

> Finished chain.


'still, tilting, and unknown'

In [25]:
df.head(3).T

,2054221,2102316,1692430
latitudeE7,337800110,338538686,413823862
longitudeE7,-843902467,-843669955,21723678
accuracy,16,65,14
source,wifi,NaN,wifi
timestamp,2019-11-29 15:04:29.611000+00:00,2020-08-23 03:36:11.015000+00:00,2018-09-09 02:43:14.494000+00:00
deviceDesignation,NaN,NaN,primary
deviceTag,1794310557.0,-1949129349.0,-2093536168.0
platformType,NaN,NaN,NaN
serverTimestamp,NaN,NaN,NaN
deviceTimestamp,NaN,NaN,NaN


## Load Semantic

In [28]:
import os

data_folder = "/Users/davidrose/git/chatlas/data/location_history/semantic/2023"
data_files = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.json')]

data = []
for file in data_files:
    with open(file, 'r') as f:
        data.append(json.load(f))

dfs = pd.json_normalize(data)


In [86]:
activity_segments = []
place_visits = []
for i in data[0]["timelineObjects"]:
    if "activitySegment" in i.keys():
        activity_segments.append(i["activitySegment"])
    if "placeVisit" in i.keys():
        place_visits.append(i["placeVisit"])

df_acts = pd.json_normalize(activity_segments)
df_visits = pd.json_normalize(place_visits)

### Make interval DF

In [87]:
def parse_datetime(dt_str):
    try:
        # Try the format with milliseconds
        return datetime.strptime(dt_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    except ValueError:
        # Fall back to the format without milliseconds
        return datetime.strptime(dt_str, '%Y-%m-%dT%H:%M:%SZ')
    
df_acts["duration.startTimestamp"] = df_acts["duration.startTimestamp"].apply(parse_datetime)
df_acts["duration.endTimestamp"] = df_acts["duration.endTimestamp"].apply(parse_datetime)

In [88]:
new_rows = []
for _, row in df_acts.iterrows():
    start_time = row["duration.startTimestamp"]
    end_time = row["duration.endTimestamp"]
    current_time = start_time
    
    while current_time < end_time:
        next_interval_end = current_time + pd.Timedelta(minutes=5)
        new_row = row.copy()
        new_row["duration.startTimestamp"] = current_time
        new_row["duration.endTimestamp"] = min(next_interval_end, end_time)
        new_rows.append(new_row)
        current_time = next_interval_end

granular_df = pd.DataFrame(new_rows)

In [89]:
granular_df.shape

(369, 35)

In [96]:
# Create an empty DataFrame that covers all 5-minute intervals for the month
start_date = df_acts["duration.startTimestamp"].min().replace(hour=0, minute=0, second=0, microsecond=0)
end_date = df_acts["duration.endTimestamp"].max().replace(hour=23, minute=59, second=59, microsecond=0)
date_range = pd.date_range(start_date, end_date, freq="5T")
all_intervals_df = pd.DataFrame(index=date_range)
all_intervals_df = all_intervals_df.reset_index().rename(columns={"index": "interval_start"})

# Merge the base DataFrame with the granular_df
merged_df = pd.merge_asof(all_intervals_df.sort_values(by="interval_start"),
                          granular_df.sort_values(by='duration.startTimestamp'),
                          left_on="interval_start",
                          right_on="duration.startTimestamp",
                          direction="backward",
                          suffixes=("", "_y"))

# Drop unwanted columns and clean up if necessary
# merged_df.drop(columns=['duration.startTimestamp_y', 'duration.endTimestamp_y'], inplace=True)

In [97]:
merged_df.shape

(8928, 36)